<a href="https://colab.research.google.com/github/Harklink/machine-learning-for-networks/blob/master/TP_Detection_corrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previously

In [ ]:
!pip install plotly==5.8.0

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import plotly
import pandas as pd
pd.options.plotting.backend = "plotly"

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1hI89q2QvdxYS4ZIYFDBTo10APircmCgL'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('timeseries.zip')
!unzip timeseries.zip
!mkdir timeseries
!mv *.csv ./timeseries/

# Situation
Des courbes très bruité, présentant des non linéarités et avec beaucoup d'épi-evenements anodins.

## Probleme
Comment trouver l'incident/les incidents ?

## Solution 1 :
On cherche a la main :) (GL HF)

## Solution 2 :
On est malin.

## Solution 3 :
Vos idées ?



# Trouver l'incident

### Piste de recherche

Clustering, series derivées, ...

In [ ]:
from sklearn.cluster import KMeans
import glob
import numpy as np
import plotly.graph_objects as go

def derivate_timeseries(df, number_of_clusters, time_window_size, feature_cols):
  figs = []
  for feature in feature_cols:
    kmeans = KMeans(n_clusters=number_of_clusters)
    df[f"cluster_{feature}"] = kmeans.fit_predict(df.loc[:, [feature]])
    df[f"clustered_{feature}_mean"] = df.groupby(f"cluster_{feature}")[feature].rolling(time_window_size, min_periods=2).mean().reset_index(0,drop=True)
    df[f"clustered_{feature}_pop"] = df.groupby(f"cluster_{feature}")[feature].rolling(time_window_size, min_periods=2).count().reset_index(0,drop=True)
    df[f"cluster_{feature}_str"] = df[f"cluster_{feature}"].astype(str)
    #figs.append(df.plot(x=df.index, y=feature_cols))
    #figs.append(df.plot.scatter(x=df.index, y=feature, color=f"cluster_{feature}_str", title=f"{site}-{app}_latencies"))
    figs.append(df.plot.line(x=df.index, y=f"clustered_{feature}_mean", color=f"cluster_{feature}_str", title=f"{site}-{app}_cluster_means"))
    figs.append(df.plot.line(x=df.index, y=f"clustered_{feature}_pop", color=f"cluster_{feature}_str", title=f"{site}-{app}_cluster_populations"))
  return figs

feature_cols = ['latency.total']
sites = ['jasin', 'shangai',]
apps = ['salesforce']
for site in sites :
  for app in apps:
    filepath = "./timeseries/{}-{}.csv".format(site,app)
    df = pd.read_csv(filepath, low_memory=False, parse_dates=["@timestamp"], keep_date_col=False, index_col="@timestamp")
    df["app"]= filepath.split('/')[2].split(".")[0]
    figs = derivate_timeseries(df, 4, "1200s", feature_cols)
    for fig in figs:
      fig.update_traces(connectgaps=False)
      fig.show()



> Methode Analyse Limites ? 